<a href="https://colab.research.google.com/github/saddarudin/google_colab/blob/main/nlp_fastText_custom_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training a model that understands Indian Recepies better

In [1]:
!pip install fastText

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fastText: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313490 sha256=35977e7ed40593c3e340a20951c68fc38010b1e91b46b3750f08013ee596441c
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fastText


In [2]:
import fasttext

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('Indian_food.csv')
print(df.shape)
df.head()

(5938, 9)


,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url,Ingredient-count
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",https://www.archanaskitchen.com/images/archana...,10
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,"tomato,salt,chickpea lentils,green chilli,rice...",https://www.archanaskitchen.com/images/archana...,12
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,"salt,rice vermicelli noodles (thin),asafoetida...",https://www.archanaskitchen.com/images/archana...,12
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",https://www.archanaskitchen.com/images/archana...,15
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seed...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"tomato,salt,ginger,red chillies,curry,asafoeti...",https://www.archanaskitchen.com/images/archana...,12


In [5]:
df.TranslatedInstructions[0]

'To begin making the Masala Karela Recipe,de-seed the karela and slice.\nDo not remove the skin as the skin has all the nutrients.\nAdd the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles.\nRelease the pressure immediately and open the lids.\nKeep aside.Heat oil in a heavy bottomed pan or a kadhai.\nAdd cumin seeds and let it sizzle.Once the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.Add the karela, red chilli powder, amchur powder, coriander powder and besan.\nStir to combine the masalas into the karela.Drizzle a little extra oil on the top and mix again.\nCover the pan and simmer Masala Karela stirring occasionally until everything comes together well.\nTurn off the heat.Transfer Masala Karela into a serving bowl and serve.Serve Masala Karela along with Panchmel Dal and Phulka for a weekday meal with your family.\n'

## Removing white spaces and special characters like slash-n

In [6]:
import re

In [7]:
# using regular expression
text = "To begin making the Masala Karela Recipe,de-seed the karela and slice.\nDo not remove the skin as the skin has all the nutrients.\nAdd the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles.\nRelease the pressure immediately and open the lids.\nKeep aside.Heat oil in a heavy bottomed pan or a kadhai.\nAdd cumin seeds and let it sizzle.Once the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.Add the karela, red chilli powder, amchur powder, coriander powder and besan.\nStir to combine the masalas into the karela.Drizzle a little extra oil on the top and mix again.\nCover the pan and simmer Masala Karela stirring occasionally until everything comes together well.\nTurn off the heat.Transfer Masala Karela into a serving bowl and serve.Serve Masala Karela along with Panchmel Dal and Phulka for a weekday meal with your family."

In [8]:
re.sub(r"[^\w\s]",' ',text,flags=re.MULTILINE)

'To begin making the Masala Karela Recipe de seed the karela and slice \nDo not remove the skin as the skin has all the nutrients \nAdd the karela to the pressure cooker with 3 tablespoon of water  salt and turmeric powder and pressure cook for three whistles \nRelease the pressure immediately and open the lids \nKeep aside Heat oil in a heavy bottomed pan or a kadhai \nAdd cumin seeds and let it sizzle Once the cumin seeds have sizzled  add onions and saute them till it turns golden brown in color Add the karela  red chilli powder  amchur powder  coriander powder and besan \nStir to combine the masalas into the karela Drizzle a little extra oil on the top and mix again \nCover the pan and simmer Masala Karela stirring occasionally until everything comes together well \nTurn off the heat Transfer Masala Karela into a serving bowl and serve Serve Masala Karela along with Panchmel Dal and Phulka for a weekday meal with your family '

In [9]:
re.sub("[ \n]+"," ","powder    amchur poweder \n corindar",flags=re.MULTILINE)

'powder amchur poweder corindar'

In [10]:
def preprocess(text):
  text = re.sub(r"[^\w\s\']",' ',text)
  text = re.sub(r"[ \n]+",' ',text)
  return text.strip().lower()

preprocess(text)

'to begin making the masala karela recipe de seed the karela and slice do not remove the skin as the skin has all the nutrients add the karela to the pressure cooker with 3 tablespoon of water salt and turmeric powder and pressure cook for three whistles release the pressure immediately and open the lids keep aside heat oil in a heavy bottomed pan or a kadhai add cumin seeds and let it sizzle once the cumin seeds have sizzled add onions and saute them till it turns golden brown in color add the karela red chilli powder amchur powder coriander powder and besan stir to combine the masalas into the karela drizzle a little extra oil on the top and mix again cover the pan and simmer masala karela stirring occasionally until everything comes together well turn off the heat transfer masala karela into a serving bowl and serve serve masala karela along with panchmel dal and phulka for a weekday meal with your family'

In [11]:
df.TranslatedInstructions = df.TranslatedInstructions.map(preprocess)
df.TranslatedInstructions[0]

'to begin making the masala karela recipe de seed the karela and slice do not remove the skin as the skin has all the nutrients add the karela to the pressure cooker with 3 tablespoon of water salt and turmeric powder and pressure cook for three whistles release the pressure immediately and open the lids keep aside heat oil in a heavy bottomed pan or a kadhai add cumin seeds and let it sizzle once the cumin seeds have sizzled add onions and saute them till it turns golden brown in color add the karela red chilli powder amchur powder coriander powder and besan stir to combine the masalas into the karela drizzle a little extra oil on the top and mix again cover the pan and simmer masala karela stirring occasionally until everything comes together well turn off the heat transfer masala karela into a serving bowl and serve serve masala karela along with panchmel dal and phulka for a weekday meal with your family'

In [12]:
df.to_csv('food_recepies.txt',columns=['TranslatedInstructions'],header=None, index=False)

In [13]:
model = fasttext.train_unsupervised('food_recepies.txt')

In [14]:
model.get_nearest_neighbors("chutney")

[(0.9275704622268677, 'chutneys'),
 (0.7463748455047607, 'dhaniya'),
 (0.7132056951522827, 'imli'),
 (0.7042087316513062, 'khajur'),
 (0.6639349460601807, 'kanchipuram'),
 (0.6590506434440613, 'pudina'),
 (0.6549491286277771, 'gothsu'),
 (0.6544407606124878, 'chammanthi'),
 (0.6525646448135376, 'south'),
 (0.6511055827140808, 'madurai')]

In [15]:
model.get_nearest_neighbors('halwa')

[(0.7467402815818787, 'khoya'),
 (0.7186369299888611, 'burfi'),
 (0.7104381322860718, 'rabri'),
 (0.6857462525367737, 'mawa'),
 (0.6752265095710754, 'badam'),
 (0.672613799571991, 'sheera'),
 (0.6673717498779297, 'kheer'),
 (0.6628114581108093, 'mohan'),
 (0.6588674187660217, 'basundi'),
 (0.6500763297080994, 'doodh')]

In [16]:
model.get_nearest_neighbors('saragva')

[(0.8921212553977966, 'fansi'),
 (0.8687644004821777, 'bhoplya'),
 (0.8530213832855225, 'phanu'),
 (0.8452553749084473, 'agathi'),
 (0.8449345827102661, 'saagu'),
 (0.8402233719825745, 'sookhi'),
 (0.8388527631759644, 'vaangi'),
 (0.8383015394210815, 'bhuga'),
 (0.8374607563018799, 'sukhi'),
 (0.8336049318313599, 'phalguni')]

In [17]:
model.get_word_vector('dosa')

array([-0.11950116,  0.28583047, -0.4654494 ,  0.08693456, -0.35507378,
        0.7289869 ,  0.20280266, -0.11838338,  0.23742546, -0.19064467,
       -0.31924918,  0.13865896, -0.6260956 , -0.6092955 , -0.23994708,
        0.23472966,  0.2875553 , -0.05696383,  0.35141674,  0.04170053,
       -0.6357446 , -0.18519005, -0.16451463,  0.16385582,  0.00592823,
       -0.02217221,  0.741354  , -0.13828816, -0.5270438 , -0.1519697 ,
       -0.20615944, -1.2354382 , -0.08859859, -0.19523752, -0.63037205,
       -0.09277713,  0.24950927, -0.00869391, -0.98898953,  0.44255862,
       -0.4225871 ,  0.02535464,  0.45536876,  0.8443987 ,  0.28843385,
       -0.08182858,  0.2496074 , -0.7621703 , -0.1465675 ,  0.02497284,
        0.06998996,  0.18063627, -0.0653295 , -0.13767588,  0.25840017,
       -0.05835231,  0.6013155 ,  0.1673816 , -0.43791327, -0.2440163 ,
       -0.18183254,  0.09999452,  0.63193876, -0.52141434, -0.17266056,
        0.33686408, -0.75262123,  0.16485716,  0.06816921, -0.31